# 1 Introduction

### 1.1 Createn Agent

#### Setup Ollama Dependencis
- Look at note on how to start ollama in the background
- window 1: ollama serve
- window 2: ollama run llama3.2

In [6]:
# Set the OLLAMA_BASE_URL environment variable
import os
ollama_url = 'http://127.0.0.1:11434' # from running ollama serve
os.environ["OLLAMA_BASE_URL"] = ollama_url

# Verify that it has been set
print(f"OLLAMA_BASE_URL is set to: {os.getenv('OLLAMA_BASE_URL')}")

OLLAMA_BASE_URL is set to: http://127.0.0.1:11434


#### Clean up previous agents

In [7]:
from letta import ChatMemory, EmbeddingConfig, LLMConfig, create_client
client = create_client()

for agent_state in client.list_agents():
    print(f'AgentId: {agent_state.id}')
    client.delete_agent(agent_id=agent_state.id)


AgentId: agent-6c2797c3-7023-4db3-be28-f53c2d288975


#### Create Client

In [8]:
from letta import ChatMemory, EmbeddingConfig, LLMConfig, create_client
from letta.prompts import gpt_system

client = create_client()

llm_config = LLMConfig(
    # llama model versions: https://github.com/ollama/ollama
    model="llama3.2:1b", #"llama3.2", #ollama run llama3.2:1b
    model_endpoint_type="ollama", # llamacp
    model_endpoint=ollama_url,
    context_window=16384
)
embedding_config = EmbeddingConfig.default_config(model_name="text-embedding-ada-002")

# create a new agent
agent_state = client.create_agent(
    # agent's name (unique per-user, autogenerated if not provided)
    name="agent_name",
    # in-context memory representation with human/persona blocks
    memory=ChatMemory(
      human="Name: Sarah", 
      persona="You are a helpful assistant that loves emojis"
    ),
    # LLM model & endpoint configuration
    llm_config=llm_config,
    # embedding model & endpoint configuration (cannot be changed)
    embedding_config=embedding_config,
    # system instructions for the agent (defaults to `memgpt_chat`)
    system=gpt_system.get_system_text("memgpt_chat"),
    # whether to include base letta tools (default: True)
    include_base_tools=True,
    # list of additional tools (by name) to add to the agent
    tools=[],
)
print(f"Created agent with name {agent_state.name} and unique ID {agent_state.id}")

Created agent with name agent_name and unique ID agent-155f1cc4-9200-4915-a6fe-d696a5d1057f


In [9]:
# get the agent by ID
agent_state = client.get_agent(agent_id=agent_state.id)

# get the agent by name
agent_id = client.get_agent_id(agent_name=agent_state.name)
agent_state = client.get_agent(agent_id=agent_id)

print(f'agent_state={agent_state}')

agent_state=description=None metadata_={'human:': 'basic', 'persona': 'sam_pov'} user_id='user-00000000-0000-4000-8000-000000000000' id='agent-155f1cc4-9200-4915-a6fe-d696a5d1057f' name='agent_name' created_at=datetime.datetime(2024, 12, 4, 22, 38, 6, 952798) message_ids=['message-e3aaf047-7beb-4984-a997-1aff64aec076', 'message-1b51e872-2130-4449-8fbf-0616a6886664', 'message-040277c0-ee61-4ba0-85eb-c26edacebbaa', 'message-7f18c55c-4d49-46e5-bc96-b698c71f480d'] memory=Memory(memory={'persona': Block(value='You are a helpful assistant that loves emojis', limit=2000, template_name=None, template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-305056d7-2ddc-4c15-8ed0-a85d663e6102'), 'human': Block(value='Name: Sarah', limit=2000, template_name=None, template=False, label='human', description=None, metadata_={}, user_id=None, id='block-b56eda67-c4e7-40fd-9f0d-b4c3e0f53ee4')}, prompt_template='{% for block in memory.values() %}<{{ block.label }} characters="{{

### 1.2 Send a message Client

In [10]:
print(1)
# Message an agent
response = client.send_message(
  agent_id=agent_state.id, 
  role="user", 
  message="hello There.  I like pickleball alot.  Tel me some paddles I can get."
)
print(2)
print("Usage", response.usage)

print(3)
print("Agent messages", response.messages)

1
2
Usage completion_tokens=76 prompt_tokens=2218 total_tokens=2279 step_count=1
3
Agent messages [InternalMonologue(id='message-444178d1-33b4-42ed-9556-c31e04325737', date=datetime.datetime(2024, 12, 5, 6, 38, 21, 835942, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="Hello, I'm Letta. Nice to meet you."), FunctionCallMessage(id='message-444178d1-33b4-42ed-9556-c31e04325737', date=datetime.datetime(2024, 12, 5, 6, 38, 21, 835942, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "Pickleball is great! You can get a variety of paddles at most sports stores."\n}', function_call_id='73de3f6a-e43f-4ef6-94b1-79d40')), FunctionReturn(id='message-61c2de2a-e6fc-4f23-be32-7f5c5a924bca', date=datetime.datetime(2024, 12, 5, 6, 38, 21, 838219, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time":

In [12]:
client.list_agents()

#client.delete_agent(agent_id=agent_state.id)

[AgentState(description=None, metadata_={'human:': 'basic', 'persona': 'sam_pov'}, user_id='user-00000000-0000-4000-8000-000000000000', id='agent-155f1cc4-9200-4915-a6fe-d696a5d1057f', name='agent_name', created_at=datetime.datetime(2024, 12, 4, 22, 38, 6, 952798), message_ids=['message-a10b87cf-f915-4ff1-954b-9b2b45bee1ed', 'message-1b51e872-2130-4449-8fbf-0616a6886664', 'message-040277c0-ee61-4ba0-85eb-c26edacebbaa', 'message-7f18c55c-4d49-46e5-bc96-b698c71f480d', 'message-68ee14e9-d619-42b6-8057-41e98531ede4', 'message-444178d1-33b4-42ed-9556-c31e04325737', 'message-61c2de2a-e6fc-4f23-be32-7f5c5a924bca'], memory=Memory(memory={'persona': Block(value='You are a helpful assistant that loves emojis', limit=2000, template_name=None, template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-305056d7-2ddc-4c15-8ed0-a85d663e6102'), 'human': Block(value='Name: Sarah', limit=2000, template_name=None, template=False, label='human', description=None, metadata_={}

# 2 Tools
- Agents map NLU to actions --> tools
- Documentation: https://docs.letta.com/agents/tools
- Letta comes with a default set of tools: **send_message, conversation_search, conversation_search_date, archival_memory_insert, archival_memory_search**

## 2.1 Define My Tool

In [15]:
def roll_d20(self) -> str:
    """
    Simulate the roll of a 20-sided die (d20).

    This function generates a random integer between 1 and 20, inclusive,
    which represents the outcome of a single roll of a d20.

    Returns:
        int: A random integer between 1 and 20, representing the die roll.

    Example:
        >>> roll_d20()
        15  # This is an example output and may vary each time the function is called.
    """
    import random

    dice_role_outcome = random.randint(1, 20)
    output_string = f"You rolled a {dice_role_outcome}"
    return output_string


from letta import create_client
if not client:
    client = create_client()

tool = client.create_tool(roll_d20)    
tool


Tool(id='tool-ab4e96ec-e51d-4120-8c2a-3ea2b30a9f1c', description=None, source_type='python', module=None, organization_id='org-00000000-0000-4000-8000-000000000000', name='roll_d20', tags=[], source_code='def roll_d20(self) -> str:\n    """\n    Simulate the roll of a 20-sided die (d20).\n\n    This function generates a random integer between 1 and 20, inclusive,\n    which represents the outcome of a single roll of a d20.\n\n    Returns:\n        int: A random integer between 1 and 20, representing the die roll.\n\n    Example:\n        >>> roll_d20()\n        15  # This is an example output and may vary each time the function is called.\n    """\n    import random\n\n    dice_role_outcome = random.randint(1, 20)\n    output_string = f"You rolled a {dice_role_outcome}"\n    return output_string\n', json_schema={'name': 'roll_d20', 'description': 'Simulate the roll of a 20-sided die (d20).', 'parameters': {'type': 'object', 'properties': {'request_heartbeat': {'type': 'boolean', 'descr

## 2.2 Create Agent with Tool

#### Delete Previous Agents

In [17]:
client = create_client()

for agent_state in client.list_agents():
    print(f'Deleting AgentId: {agent_state.id}')
    client.delete_agent(agent_id=agent_state.id)


#### Create Agent With My Tool
- Tool Rules: https://docs.letta.com/agents/tools
    * Defines behaviors of tools, ie when it is called and what happens after it's called 

In [18]:
from letta.schemas.tool_rule import TerminalToolRule
from letta import EmbeddingConfig, LLMConfig

llm_config = LLMConfig(
    # llama model versions: https://github.com/ollama/ollama
    model="llama3.2:1b", #"llama3.2", #ollama run llama3.2:1b
    model_endpoint_type="ollama", # llamacp
    model_endpoint=ollama_url,
    context_window=16384
)
embedding_config = EmbeddingConfig.default_config(model_name="text-embedding-ada-002")

# create a new agent
agent_state = client.create_agent(
    # agent's name (unique per-user, autogenerated if not provided)
    #name="agent_name",
    
    # in-context memory representation with human/persona blocks
    #memory=ChatMemory(
    #  human="Name: Sarah", 
    #  persona="You are a helpful assistant that loves emojis"
    #),
    
    # LLM model & endpoint configuration
    llm_config=llm_config,
    
    # embedding model & endpoint configuration (cannot be changed)
    embedding_config=embedding_config,
    
    # system instructions for the agent (defaults to `memgpt_chat`)
    #system=gpt_system.get_system_text("memgpt_chat"),
    
    # whether to include base letta tools (default: True)
    # include_base_tools=True,

    # list of additional tools (by name) to add to the agent
    tools=[tool.name],

    # add tool rules that terminate execution after specific tools
    tool_rules=[
        # exit after roll_d20 is called
        TerminalToolRule(tool_name=tool.name),
        # exit after send_message is called (default behavior)
        TerminalToolRule(tool_name="send_message"),
    ],
)
print(f"Created agent with name {agent_state.name} with tools {agent_state.tools}")


Created agent with name OpulentStarfish with tools ['roll_d20', 'send_message', 'conversation_search', 'conversation_search_date', 'archival_memory_insert', 'archival_memory_search', 'core_memory_append', 'core_memory_replace']


#### Send a message to trigger my tool

In [19]:
# Message an agent
response = client.send_message(agent_id=agent_state.id, role="user", message="Roll the dice")
print("Usage", response.usage)

print("")
print("Agent messages", response.messages)

Usage completion_tokens=60 prompt_tokens=2508 total_tokens=2550 step_count=1

Agent messages [InternalMonologue(id='message-4c4839aa-981b-495d-bfb8-c707a3d7f7fc', date=datetime.datetime(2024, 12, 5, 6, 40, 38, 230151, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Rolling a d20... The result is: 14.'), FunctionCallMessage(id='message-4c4839aa-981b-495d-bfb8-c707a3d7f7fc', date=datetime.datetime(2024, 12, 5, 6, 40, 38, 230151, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='roll_d20', arguments='{\n  "request_heartbeat": true\n}', function_call_id='267c02eb-da92-4e27-8ef7-574f2')), FunctionReturn(id='message-825df9aa-6e6c-41c7-be5a-08988441493d', date=datetime.datetime(2024, 12, 5, 6, 40, 38, 233546, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "You rolled a 1",\n  "time": "2024-12-04 10:40:38 PM PST-0800"\n}', status='success', funct